In [ ]:
pip install -U datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 18.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; pl

In [ ]:
import os
import string
import random
import torch
import torch.nn as nn
import torch.optim as optim
from argparse import ArgumentParser, ArgumentDefaultsHelpFormatter
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import os
import unicodedata
import random
from datasets import load_dataset
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from collections import Counter
from tqdm import tqdm
import unicodedata
import random
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from collections import Counter
from tqdm import tqdm
import unicodedata
import random
from torch.utils.data import DataLoader, Dataset, TensorDataset

In [ ]:
class CharTransformer(nn.Module):
    def __init__(self, vocab_size, embedding_dim=128, num_heads=4, hidden_dim=256, num_layers=2, max_len=100):
        super(CharTransformer, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.pos_embedding = nn.Parameter(torch.zeros(1, max_len, embedding_dim))

        encoder_layer = nn.TransformerEncoderLayer(d_model=embedding_dim, nhead=num_heads, dim_feedforward=hidden_dim,dropout=0.1, batch_first=True)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

        self.fc = nn.Linear(embedding_dim, vocab_size)

    def forward(self, x):
        seq_len = x.size(1)
        x = self.embedding(x) + self.pos_embedding[:, :seq_len, :]
        out = self.transformer(x)
        logits = self.fc(out[:, -1, :])  # Use the last token's output for prediction
        return logits, None

In [ ]:
class MyModel:
    """
    This is a starter model to get you started. Feel free to modify this file.
    """
    vocab = sorted(set(string.printable))
    char2idx = {ch: idx for idx, ch in enumerate(vocab)}
    idx2char = {idx: ch for ch, idx in char2idx.items()}
    vocab_size = len(vocab)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    def __init__(self,char2idx, idx2char, vocab_size):
        #self.model = CharLSTM(self.vocab_size).to(self.device)
        self.char2idx = char2idx
        self.idx2char = idx2char
        self.vocab_size = vocab_size
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model = CharTransformer(self.vocab_size).to(self.device)

    def load_oscar_char_data(
      lang_code='en',
      seq_len=30,
      sample_size=10000,
      pair_fraction=0.2,
      test_split=0.1,
      top_k_chars=2000
    ):
      """
      Stream OSCAR dataset (for a specific language), normalize it, and return character-level training pairs.
      """
      print(f"🔄 Streaming OSCAR ({lang_code}) from Hugging Face Hub...")
      dataset = load_dataset(
          "oscar",
          f"unshuffled_deduplicated_{lang_code}",
          split="train",
          streaming=True
      )

      def normalize(text):
          text = unicodedata.normalize("NFKC", text)
          return ''.join(c for c in text if not unicodedata.category(c).startswith('C'))

      print("📏 Collecting and normalizing text...")
      buffer = ""
      count = 0
      for sample in tqdm(dataset, total=sample_size*2):
          raw = sample.get("text", "").strip()
          if raw:
              buffer += normalize(raw) + "\n"
              count += 1
          if count >= sample_size:
              break

      print(f"✅ Collected text: {len(buffer):,} characters from {count} samples")

      # Build character vocabulary
      counter = Counter(buffer)
      vocab = [c for c, _ in counter.most_common(top_k_chars)]
      char2idx = {ch: i for i, ch in enumerate(vocab)}
      idx2char = {i: ch for ch, i in char2idx.items()}

      # Generate (context, next_char) pairs
      num_possible = len(buffer) - seq_len
      num_pairs = int(num_possible * pair_fraction)
      if num_pairs < 1:
          num_pairs = 1

      indices = random.sample(range(num_possible), num_pairs)
      data = []
      for i in tqdm(indices, desc="🔗 Generating training pairs"):
          context = buffer[i:i+seq_len]
          next_char = buffer[i+seq_len]
          if all(c in char2idx for c in context + next_char):
              data.append((context, next_char))

      print(f"🟢 Total training pairs: {len(data)}")
      train_data, _ = train_test_split(data, test_size=test_split, random_state=42)

      return train_data, vocab, char2idx, idx2char

    @classmethod
    def load_test_data(cls, fname, sequence_length=10):
        # your code here
        data = []
        with open(fname) as f:
            for line in f:
                line = line.strip()
                #inp = line[:-1]  # the last character is a newline
                #data.append(inp)
                for i in range(len(line) - sequence_length):
                    context = line[i:i+sequence_length]
                    next_char = line[i+sequence_length]
                    if all(c in cls.char2idx for c in context + next_char):
                        data.append((context, next_char))
        return data

    @classmethod
    def write_pred(cls, preds, fname):
        with open(fname, 'wt') as f:
            for p in preds:
                f.write('{}\n'.format(p))

    def run_train(self, data, work_dir, epochs=3, lr=0.003, batch_size=64):
      # Prepare tensors
      X = torch.tensor([[self.char2idx[c] for c in context] for context, _ in data], dtype=torch.long)
      y = torch.tensor([self.char2idx[target] for _, target in data], dtype=torch.long)

      dataset = TensorDataset(X, y)
      dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

      # Loss and optimizer
      criterion = nn.CrossEntropyLoss()
      optimizer = optim.Adam(self.model.parameters(), lr=lr)

      print('Training on {} samples'.format(len(data)))
      print('Using device {}'.format(self.device))
      self.model.train()

      for epoch in range(epochs):
          print(f"Epoch {epoch+1}/{epochs}")
          total_loss = 0.0

          for x_batch, y_batch in tqdm(dataloader, desc=f"Epoch {epoch+1}"):
              x_batch = x_batch.to(self.device)
              y_batch = y_batch.to(self.device)

              optimizer.zero_grad()
              logits, _ = self.model(x_batch)
              loss = criterion(logits, y_batch)
              loss.backward()

              # 🚨 Gradient clipping (important for LSTM)
              torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=1.0)

              optimizer.step()
              total_loss += loss.item() * x_batch.size(0)  # Multiply by batch size for total

          avg_loss = total_loss / len(dataset)
          print(f"Epoch {epoch+1}/{epochs} - Avg Loss: {avg_loss:.4f}")

      # Save model
      torch.save({
          'model_state_dict': self.model.state_dict(),
          'char2idx': self.char2idx,
          'idx2char': self.idx2char,
          'vocab_size': self.vocab_size
      }, os.path.join(work_dir, 'model.pt'))


    def run_pred(self, data):
        # your code here
        self.model.eval()
        preds = []
        all_chars = string.ascii_letters
        with torch.no_grad():
            for context, _ in data:
                x = torch.tensor([[self.char2idx.get(c, 0) for c in context]], dtype=torch.long).to(self.device)
                logits, _ = self.model(x)
                probs = torch.softmax(logits, dim=-1)
                top3 = torch.topk(probs, 3).indices[0].tolist()
                pred_chars = [self.idx2char[i] for i in top3]
                preds.append(''.join(pred_chars))
        return preds
        #for inp in data:
         #   # this model just predicts a random character each time
          #  top_guesses = [random.choice(all_chars) for _ in range(3)]
           # preds.append(''.join(top_guesses))
        #return preds

    def save(self, work_dir):
        # your code here
        # this particular model has nothing to save, but for demonstration purposes we will save a blank file
        with open(os.path.join(work_dir, 'model.checkpoint'), 'wt') as f:
            f.write('dummy save')

    @classmethod
    def load(cls, work_dir):
        # your code here
        # this particular model has nothing to load, but for demonstration purposes we will load a blank file
        #with open(os.path.join(work_dir, 'model.checkpoint')) as f:
         #   dummy_save = f.read()
        #return MyModel()
        model = cls()
        checkpoint = torch.load(os.path.join(work_dir, 'model.pt'), map_location=cls.device)
        model.model.load_state_dict(checkpoint['model_state_dict'])
        model.model.to(cls.device)
        model.model.eval()
        return model

In [ ]:
print('Instatiating model')

print('Loading training data')
#train_data = MyModel.load_training_data()
#train_data, vocab, char2idx, idx2char = MyTransformerModel.load_oscar_char_data()
train_data, vocab, char2idx, idx2char = MyModel.load_oscar_char_data(

)
model = MyModel(char2idx, idx2char, len(vocab))
print('Training')
model.run_train(train_data, './work')
print('Saving model')
model.save('./work')

Instatiating model
Loading training data
🔄 Streaming OSCAR (en) from Hugging Face Hub...
📏 Collecting and normalizing text...


 50%|████▉     | 9999/20000 [00:16<00:16, 601.84it/s]


✅ Collected text: 48,337,613 characters from 10000 samples


🔗 Generating training pairs: 100%|██████████| 9667516/9667516 [00:27<00:00, 348380.17it/s]


🟢 Total training pairs: 9665205
Training
Training on 8698684 samples
Using device cuda
Epoch 1/3


...

Epoch 1/3 - Avg Loss: 2.5234
Epoch 2/3


Epoch 2: 100%|██████████| 135917/135917 [16:11<00:00, 139.87it/s]


Epoch 2/3 - Avg Loss: 2.4392
Epoch 3/3


Epoch 3: 100%|██████████| 135917/135917 [16:12<00:00, 139.81it/s]

Epoch 3/3 - Avg Loss: 2.3538
Saving model
